In [3]:
from typing import List, Dict, Optional, Set, Tuple
import logging
import gspread
from dotenv import load_dotenv
from typing import Optional, List
from sheet_manager.sheet_crud.sheet_crud import SheetManager

# Initialize SheetManager
sheet_manager = SheetManager()

# Debugging the update_cell_by_condition function
row_updated = sheet_manager.update_cell_by_condition(
    condition_column="huggingface_id", 
    condition_value="dd", 
    target_column="benchmark_name", 
    target_value="ddd"
)

Updated row 3: Set benchmark_name to 'ddd' where huggingface_id is 'dd'


In [2]:
from sheet_manager.sheet_checker.sheet_check import SheetChecker
from sheet_manager.sheet_crud.sheet_crud import SheetManager
sheet_manager = SheetManager()

checker = SheetChecker(sheet_manager)
model_added, benchmark_exists = checker.check_model_and_benchmark("test-model", "COCO")

if model_added:
    print("새로운 모델이 추가되었습니다")
if not benchmark_exists:
    print("벤치마크 측정이 필요합니다")

Successfully switched to column: Model name
Successfully switched to column: Model name
Successfully switched to column: COCO
벤치마크 측정이 필요합니다


In [1]:
import tempfile
import shutil
import os
import logging
from typing import List, Dict, Optional, Tuple
from pathlib import Path
import json
import numpy as np
from pia_bench.checker.bench_checker import BenchChecker

with tempfile.TemporaryDirectory() as temp_dir:
    # 테스트 디렉토리 구조 생성
    base_path = Path(temp_dir)
    benchmark_name = "test_benchmark"
    model_name = "test_model"
    cfg_prompt = "topk"
    
    # 필요한 디렉토리 생성
    (base_path / benchmark_name / "dataset" / "category1").mkdir(parents=True)
    (base_path / benchmark_name / "CFG").mkdir(parents=True)
    (base_path / benchmark_name / "models" / model_name / "CFG" / cfg_prompt / "metrics").mkdir(parents=True)
    (base_path / benchmark_name / "models" / model_name / "vector" / "video").mkdir(parents=True)
    
    # 테스트 파일 생성
    # 비디오 파일
    (base_path / benchmark_name / "dataset" / "category1" / "video1.mp4").touch()
    # CFG 파일
    with open(base_path / benchmark_name / "CFG" / f"{cfg_prompt}.json", "w") as f:
        json.dump({}, f)
    # 벡터 파일
    np.save(base_path / benchmark_name / "models" / model_name / "vector" / "video" / "video1.npy", np.array([1, 2, 3]))
    
    # BenchChecker 초기화 및 테스트
    checker = BenchChecker(str(base_path))
    
    print("\n테스트 1: metrics 파일이 없는 경우")
    status = checker.check_benchmark(benchmark_name, model_name, cfg_prompt)
    result = checker.get_benchmark_status(status)
    print(f"Status: {status}")
    print(f"Result: {result}")
    
    print("\n테스트 2: metrics 파일 추가 후")
    # metrics 파일 추가
    metrics_path = base_path / benchmark_name / "models" / model_name / "CFG" / cfg_prompt / "metrics" / "overall_metrics.json"
    with open(metrics_path, "w") as f:
        json.dump({}, f)
        
    status = checker.check_benchmark(benchmark_name, model_name, cfg_prompt)
    result = checker.get_benchmark_status(status)
    print(f"Status: {status}")
    print(f"Result: {result}")

INFO:pia_bench.checker.bench_checker:Found benchmark directory: test_benchmark
INFO:pia_bench.checker.bench_checker:Found 1 videos in test_benchmark dataset
INFO:pia_bench.checker.bench_checker:Found model directory: test_model
INFO:pia_bench.checker.bench_checker:Found benchmark CFG file: topk.json
INFO:pia_bench.checker.bench_checker:Found model CFG directory: topk
INFO:pia_bench.checker.bench_checker:Vector status: videos=1, vectors=1
ERROR:pia_bench.checker.bench_checker:Overall metrics file not found for test_model
INFO:pia_bench.checker.bench_checker:Found benchmark directory: test_benchmark
INFO:pia_bench.checker.bench_checker:Found 1 videos in test_benchmark dataset
INFO:pia_bench.checker.bench_checker:Found model directory: test_model
INFO:pia_bench.checker.bench_checker:Found benchmark CFG file: topk.json
INFO:pia_bench.checker.bench_checker:Found model CFG directory: topk
INFO:pia_bench.checker.bench_checker:Vector status: videos=1, vectors=1
ERROR:pia_bench.checker.bench_ch


테스트 1: metrics 파일이 없는 경우
Status: {'benchmark_exists': True, 'model_exists': True, 'cfg_files_exist': True, 'vectors_match': True, 'metrics_exist': False}
Result: no_metrics

테스트 2: metrics 파일 추가 후
Status: {'benchmark_exists': True, 'model_exists': True, 'cfg_files_exist': True, 'vectors_match': True, 'metrics_exist': False}
Result: no_metrics


In [2]:
base_path = "assets"
benchmark_name = "PIA"
model_name = "T2V_CLIP4CLIP_MSRVTT"
cfg_prompt = "topk"
checker = BenchChecker(str(base_path))
status = checker.check_benchmark(benchmark_name, model_name, cfg_prompt)
result = checker.get_benchmark_status(status)
print(f"Status: {status}")
print(f"Result: {result}")

INFO:pia_bench.checker.bench_checker:Found benchmark directory: PIA
INFO:pia_bench.checker.bench_checker:Found 94 videos in PIA dataset
INFO:pia_bench.checker.bench_checker:Found model directory: T2V_CLIP4CLIP_MSRVTT
INFO:pia_bench.checker.bench_checker:Found benchmark CFG file: topk.json
INFO:pia_bench.checker.bench_checker:Found model CFG directory: topk
INFO:pia_bench.checker.bench_checker:Vector status: videos=94, vectors=94
INFO:pia_bench.checker.bench_checker:Found overall metrics file for T2V_CLIP4CLIP_MSRVTT


Status: {'benchmark_exists': True, 'model_exists': True, 'cfg_files_exist': True, 'vectors_match': True, 'metrics_exist': True}
Result: all_passed
